In [1]:
%reset -f
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%config Completer.use_jedi = False

In [2]:
import sys
sys.path.insert(0,"..")
from profiler import *
from utils import *
from datasets import *
from models import *
from transforms import *
import torch.nn as nn
import torch.optim as optim
import torchaudio
from torch.utils.data import Dataset, ConcatDataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms
torchaudio.set_audio_backend('soundfile')

Available workers: 16


# Raspberry-Pi Data

In [3]:
model = DrosophilaNetRAW(outputs=2, dropout_p=.2)
optimizer = optim.Adam(model.parameters(), lr=0.001, eps=1e-3, amsgrad=True)
model, optimizer = load_checkpoint(f'{BASE_DATACREATEDDIR}/model_best_time_signals.pth.tar', model, optimizer)
model = model.to('cuda', dtype=torch.float)

In [4]:
dspath = "/home/kalfasyan/Dropbox/kuleuven/wingbeats_20211129/wingbeats_correct_dates/Melanogaster_202111_RPi"
#"/home/kalfasyan/data/wingbeats/Flytrap_rpi/Backup_rpi_freeflight_20210328/20210328/wingbeats"
#44100#"/home/kalfasyan/data/wingbeats/twin_cages_experiment/sensorPi/freeflight"
#48000#"/home/kalfasyan/data/wingbeats/Flytrap_rpi/Backup_rpi_freeflight_20210328/20210402/wingbeats"
#48000#"/home/kalfasyan/data/wingbeats/Flytrap_rpi/Backup_rpi_freeflight_20210328/20210328/wingbeats"
wbpf = WingbeatDatasetProfiler(dsname=dspath, custom_label=[0], rpiformat=True)

maxpeaks = 15
minpeaks = 1
maxscore = 12
minscore = 3.5
selection = f"peaks>{minpeaks} & peaks<{maxpeaks} & score>{minscore} & score<={maxscore}"
wbpf.df = wbpf.df.query(selection)
# wbpf.plot_random_wbts()
# print(wbpf.df.shape[0])

evaluate_data(wbpf.df, model)

Found 127 in dataset: /home/kalfasyan/Dropbox/kuleuven/wingbeats_20211129/wingbeats_correct_dates/Melanogaster_202111_RPi, and 1 label(s): ['wingbeats_correct_dates']
Label(s) changed to [0]


Creating a pandas Dataframe with file-paths, clean-scores, duration, sums of abs values, indice and labels..


Testing..	:   0%|          | 0/3 [00:00<?, ?it/s]

Finished.


Testing..	: 100%|██████████| 3/3 [00:01<00:00,  2.41it/s]

Accuracy: 1.18
Balanced accuracy: 1.18
Confusion matrix: 
[[0.01176471 0.98823529]
 [0.         0.        ]]
1    84
0     1
dtype: int64



/home/kalfasyan/anaconda3/envs/wbai/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1850: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


# SD card data

In [17]:
%reset -f
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%config Completer.use_jedi = False

Available workers: 16
Available workers: 16


In [18]:
import sys
sys.path.insert(0,"..")
from profiler import *
from utils import *
from datasets import *
from models import *
from transforms import *
import torch.nn as nn
import torch.optim as optim
import torchaudio
from torch.utils.data import Dataset, ConcatDataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms
torchaudio.set_audio_backend('soundfile')

In [19]:
model = DrosophilaNetRAW(outputs=2, dropout_p=.2)
optimizer = optim.Adam(model.parameters(), lr=0.001, eps=1e-3, amsgrad=True)
model, optimizer = load_checkpoint(f'{BASE_DATACREATEDDIR}/model_best_time_signals.pth.tar', model, optimizer)
model = model.to('cuda', dtype=torch.float)

In [20]:
dspath = "/home/kalfasyan/Dropbox/kuleuven/wingbeats_20211129/wingbeats_correct_dates/Melanogaster_202111_SD"
wbpf = WingbeatDatasetProfiler(dsname=dspath, custom_label=[0], rpiformat=False)

maxpeaks = 15
minpeaks = 1
maxscore = 12
minscore = 3.5
selection = f"peaks>{minpeaks} & peaks<{maxpeaks} & score>{minscore} & score<={maxscore}"
wbpf.df = wbpf.df.query(selection)

Found 11136 in dataset: /home/kalfasyan/Dropbox/kuleuven/wingbeats_20211129/wingbeats_correct_dates/Melanogaster_202111_SD, and 1 label(s): ['wingbeats_correct_dates']
Label(s) changed to [0]


Creating a pandas Dataframe with file-paths, clean-scores, duration, sums of abs values, indice and labels..
Finished.


In [21]:
evaluate_data(wbpf.df,model)

Testing..	: 100%|██████████| 185/185 [00:05<00:00, 30.95it/s]

Accuracy: 60.61
Balanced accuracy: 60.61
Confusion matrix: 
[[0.60613248 0.39386752]
 [0.         0.        ]]
0    3578
1    2325
dtype: int64



/home/kalfasyan/anaconda3/envs/wbai/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1850: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


In [22]:
extra_df = pd.read_parquet('../data_created/suzmel_extra.parquet')
evaluate_data(extra_df, model)

Testing..	: 100%|██████████| 711/711 [00:22<00:00, 32.09it/s]


Accuracy: 92.97
Balanced accuracy: 92.42
Confusion matrix: 
[[0.91808874 0.08191126]
 [0.06972001 0.93027999]]
1    20164
0     2580
dtype: int64


# UMAP

In [89]:
import umap